<a href="https://colab.research.google.com/github/kemwe/LLM/blob/master/BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nvcc4jupyter
!pip install --upgrade diffusers
!pip install transformers accelerate safetensors
!pip install plotly_express

%load_ext nvcc4jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 42.9 MB/s eta 0:00:00
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp6um8y3hx".


In [2]:
from transformers import BertForQuestionAnswering, BertTokenizerFast
import pandas as pd
from scipy.special import softmax
import torch
import numpy as np
import plotly_express as px

In [3]:
context = "The giraffe is a large African hoofed mammal belonging to the genus Giraffa. It is the tallest living terrestrial animal and the largest ruminant on Earth. Traditionally, giraffes were thought to be one species, Giraffa camelopardalis, with nine subspecies. Most recently, researchers proposed dividing them into up to eight extant species due to new research into their mitochondrial and nuclear DNA, as well as morphological measurements. Seven other extinct species of Giraffa are known from the fossil record."
question = "How many giraffe species are there?"


In [4]:
model_name = "deepset/bert-base-cased-squad2"

In [5]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
inputs = tokenizer(question, context, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1731,  1242,   176,  5132, 15475,  1530,  1132,  1175,   136,
           102,  1109,   176,  5132, 15475,  1110,   170,  1415,  2170, 16358,
         10008,  1174, 12477, 27568,  7078,  1106,  1103,  2804,   144,  5132,
          3101,  1161,   119,  1135,  1110,  1103, 14369,  1690, 13669,  3724,
          1105,  1103,  2026,   187, 14088, 14618,  1113,  2746,   119, 19324,
           117,   176,  5132, 15475,  1116,  1127,  1354,  1106,  1129,  1141,
          1530,   117,   144,  5132,  3101,  1161,  1338, 13200,  2881, 14200,
           117,  1114,  2551, 11332,   119,  2082,  3055,   117,  6962,  3000,
         18699,  1172,  1154,  1146,  1106,  2022, 13551,  1530,  1496,  1106,
          1207,  1844,  1154,  1147, 27850,  1105,  4272,  5394,   117,  1112,
          1218,  1112,   182,  1766, 26920, 12307,   119,  5334,  1168,  8256,
          1530,  1104,   144,  5132,  3101,  1161,  1132,  1227,  1121,  1103,
         11671,  1647,   119,   102]])

In [7]:
with torch.no_grad():
  outputs = model(**inputs)

outputs.start_logits


tensor([[-0.9139, -7.7962, -8.3869, -7.5360, -8.5541, -8.8549, -9.0475, -8.3716,
         -8.2640, -8.1387, -8.8576, -8.1861, -4.4297, -8.2562, -6.6420, -8.3702,
         -6.6616, -5.6400, -7.4080, -6.6517, -8.3490, -9.0923, -6.9325, -7.5530,
         -8.1860, -8.7357, -8.3855, -8.0210, -6.9378, -8.5912, -7.8170, -8.9748,
         -8.3172, -6.2812, -8.1089, -7.4539, -5.9300, -8.0558, -7.2870, -8.6672,
         -8.7621, -7.9044, -7.0835, -7.4917, -8.5977, -9.4810, -8.8991, -7.8585,
         -8.5769, -7.3498, -7.9248, -2.8019, -7.8767, -7.4992, -8.2687, -8.1664,
         -7.1814, -7.9879, -8.0986, -2.2048, -6.3627, -9.1980, -4.6610, -7.8274,
         -6.6942, -8.4344, -5.8655, -8.5037, -8.3164, -9.6530, -9.0900, -7.5385,
         -1.6094, -5.4381, -6.0882, -6.2405, -7.8734, -8.5338, -7.3541, -7.6693,
         -7.3832, -6.9873, -6.8303, -0.7111, -6.6180,  0.8172, -5.6891, -5.0849,
         -7.6422, -8.8943, -8.0146, -8.0222, -8.7827, -8.5870, -6.3772, -9.0199,
         -7.8413, -8.4475, -

In [8]:
outputs.end_logits


tensor([[-0.4806, -8.2022, -7.6392, -9.0628, -8.4349, -7.7108, -7.4595, -8.3333,
         -8.1486, -8.0085, -7.9546, -8.3258, -8.0218, -8.3786, -6.7444, -8.1608,
         -8.1794, -7.2772, -7.4763, -8.1544, -7.5623, -7.7116, -8.3544, -5.7039,
         -8.4061, -8.2509, -8.4592, -8.0749, -7.8989, -8.3014, -7.0467, -5.4346,
         -6.2067, -8.1114, -8.4385, -8.7253, -5.8323, -7.7271, -7.0405, -6.8734,
         -8.1864, -8.8899, -7.2584, -8.6657, -8.4944, -6.9539, -8.1062, -5.3731,
         -6.3786, -8.5769, -8.2747, -8.0214, -8.6014, -7.9856, -6.9756, -8.6118,
         -9.1693, -8.8253, -8.5341, -2.6006, -2.7653, -7.1162, -6.8631, -8.3140,
         -6.5338, -5.4564, -7.8968, -8.2203, -7.5683, -6.0337, -6.2246, -8.8614,
         -1.6589, -2.8559, -2.2291, -8.5956, -8.0036, -8.2671, -8.6620, -8.7622,
         -8.9020, -6.5017, -8.2507, -3.8154, -7.1207,  1.1880, -4.3028, -0.4578,
         -7.9408, -8.3290, -8.8365, -8.0221, -8.3886, -8.5757, -7.2323, -8.1993,
         -8.1319, -6.9411, -

In [9]:
#tokenizer.tokenize(context)

In [10]:
start_scores, end_scores = softmax(outputs.start_logits)[0], softmax(outputs.end_logits)[0]

In [11]:
# Creating a dataframe with the scores and plotting them
scores_df = pd.DataFrame({
    "Token Position": list(range(len(start_scores))) * 2,
    "Score": list(start_scores) + list(end_scores),
    "Score Type": ["Start"] * len(start_scores) + ["End"] * len(end_scores),
})
px.bar(scores_df, x="Token Position", y="Score", color="Score Type", barmode="group", title="Start and End Scores for Tokens")

In [12]:
start_idx = np.argmax(start_scores)

In [13]:
print(start_idx)

85


In [14]:
end_idx = np.argmax(end_scores)

In [15]:
answer_ids = inputs.input_ids[0][start_idx:end_idx+1]

In [16]:
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
answer = tokenizer.convert_tokens_to_string(answer_tokens)

In [17]:
answer

'eight'

In [18]:
confidence_score = (start_scores[start_idx] + end_scores[end_idx]) / 2
confidence_score

0.6316297054290771

In [31]:
def predict_answer( context,question):
  #input = tokenizer.tokenize(question, return_tensors='pt')
  inputs = tokenizer(question, context, return_tensors="pt",max_length=512, truncation=True)

  with torch.no_grad():
    outputs = model(**inputs)

  start_scores, end_scores = softmax(outputs.start_logits)[0], softmax(outputs.end_logits)[0]

  start_idx = np.argmax(start_scores)
  end_idx = np.argmax(end_scores)

  confidence_score = (start_scores[start_idx] + end_scores[end_idx]) / 2

  answer_ids = inputs.input_ids[0][start_idx:end_idx+1]
  answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
  answer = tokenizer.convert_tokens_to_string(answer_tokens)

  if answer != tokenizer.cls_token:
    return answer, confidence_score
  return None, confidence_score

In [24]:
context = """ Coffee is a beverage brewed from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due to its caffeine content. It has the highest sales in the world market for hot drinks.[2]

Coffee production begins when the seeds of the Coffea plant's fruits (coffee cherries) are separated to produce unroasted green coffee beans. The beans are roasted and then ground into fine particles. Coffee is brewed from the ground roasted beans, which are typically steeped in hot water before being filtered out. It is usually served hot, although chilled or iced coffee is common. Coffee can be prepared and presented in a variety of ways (e.g., espresso, French press, caffè latte, or already-brewed canned coffee). Sugar, sugar substitutes, milk, and cream are often added to mask the bitter taste or enhance the flavor. There are also various coffee substitutes.

Though coffee is now a global commodity, it has a long history tied closely to food traditions around the Red Sea. The earliest credible evidence of coffee drinking as the modern beverage appears in modern-day Yemen in southern Arabia in the middle of the 15th century in Sufi shrines, where coffee seeds were first roasted and brewed in a manner similar to how it is now prepared for drinking.[3] The coffee beans were procured by the Yemenis from the Ethiopian Highlands via coastal Somali intermediaries, and cultivated in Yemen. By the 16th century, the drink had reached the rest of the Middle East and North Africa, later spreading to Europe.

The two most commonly grown coffee bean types are C. arabica and C. robusta.[4] Coffee plants are cultivated in over 70 countries, primarily in the equatorial regions of the Americas, Southeast Asia, the Indian subcontinent, and Africa. Green, unroasted coffee is traded as an agricultural commodity. The global coffee industry is massive and worth $495.50 billion as of 2023.[5] In the same year, Brazil was the leading grower of coffee beans, producing 35% of the world's total, followed by Vietnam and Colombia. While coffee sales reach billions of dollars annually worldwide, coffee farmers disproportionately live in poverty.
 Critics of the coffee industry have also pointed to its negative impact on the environment and the clearing of land for coffee-growing and water use."""

In [33]:
predict_answer(context,"What is coffee?")

('a beverage brewed from roasted coffee beans', 0.7684159278869629)

In [35]:
predict_answer(context,"What are the most common coffee beans?")

('C. arabica and C. robusta', 0.8978558778762817)

In [36]:
sentences = context.split('\n')

In [ ]:
def chunk_sentences(sentences,chuck_size,stride):
  chunks =[]

  for i in range(0,num_sentences, chunk_size - stride):
    chunk = sentences[i:i+chunk_size]
    chunks.append(chunk)

  return chunks

In [37]:
sentences

[' Coffee is a beverage brewed from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due to its caffeine content. It has the highest sales in the world market for hot drinks.[2]',
 '',
 "Coffee production begins when the seeds of the Coffea plant's fruits (coffee cherries) are separated to produce unroasted green coffee beans. The beans are roasted and then ground into fine particles. Coffee is brewed from the ground roasted beans, which are typically steeped in hot water before being filtered out. It is usually served hot, although chilled or iced coffee is common. Coffee can be prepared and presented in a variety of ways (e.g., espresso, French press, caffè latte, or already-brewed canned coffee). Sugar, sugar substitutes, milk, and cream are often added to mask the bitter taste or enhance the flavor. There are also various coffee substitutes.",
 '',
 'Though coffee is now a global commodity, it has a long history